# 🚀 ITOS - Intraday Trading Operating System

**Complete single-file Colab notebook for strategy development and testing**

**Features:**
- ✅ Complete ITOS system in one notebook
- ✅ 3 example strategies ready to run
- ✅ Real data support (Parquet files)
- ✅ Performance analytics and visualization
- ✅ Google Colab optimized (no import issues)

**How to use:**
1. Run all cells in order
2. Upload your Parquet files to `/content/drive/MyDrive/itos_data/`
3. Modify strategies or add new ones
4. Analyze results and performance

## 📋 Setup & Dependencies

In [ ]:
# Install required packages
!pip install pandas numpy pyarrow matplotlib seaborn -q

# Core imports
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
from typing import List, Dict, Optional, Tuple, Any
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

# Google Drive setup
from google.colab import drive
drive.mount('/content/drive')

# Setup paths
import os
BASE_PATH = '/content/drive/MyDrive/itos_data'
os.makedirs(BASE_PATH, exist_ok=True)
os.makedirs(f'{BASE_PATH}/parquet_files', exist_ok=True)
os.makedirs(f'{BASE_PATH}/results', exist_ok=True)

print(f"✅ Setup complete! Base path: {BASE_PATH}")

## 🏗️ Core ITOS Classes

In [ ]:
@dataclass
class Trade:
    """Represents a single trade execution."""
    symbol: str
    strategy: str
    entry_time: datetime
    exit_time: Optional[datetime]
    entry_price: float
    exit_price: Optional[float]
    quantity: int
    side: str  # 'LONG' or 'SHORT'
    pnl: Optional[float] = None
    exit_reason: Optional[str] = None
    
    def __post_init__(self):
        if self.pnl is None and self.exit_price is not None:
            if self.side == 'LONG':
                self.pnl = (self.exit_price - self.entry_price) * self.quantity
            else:
                self.pnl = (self.entry_price - self.exit_price) * self.quantity


@dataclass
class StrategyResult:
    """Results from running a strategy across all symbols."""
    strategy_name: str
    total_trades: int
    winning_trades: int
    losing_trades: int
    total_pnl: float
    max_drawdown: float
    profit_factor: float
    sharpe_ratio: float
    avg_trade_duration: float  # in minutes
    metadata: Dict[str, Any]


@dataclass
class MarketData:
    """Market data container for a single symbol."""
    symbol: str
    data: pd.DataFrame  # OHLCV with datetime index
    
    def validate(self) -> bool:
        """Validate data format and completeness."""
        required_columns = ['open', 'high', 'low', 'close', 'volume']
        if not all(col in self.data.columns for col in required_columns):
            return False
        
        # Check for missing data
        if self.data.isnull().any().any():
            return False
            
        # Check chronological order
        if not self.data.index.is_monotonic_increasing:
            return False
            
        return True


@dataclass
class StrategyConfig:
    """Configuration for a trading strategy."""
    name: str
    side: str  # 'LONG', 'SHORT', 'BOTH'
    entry_time_start: time
    entry_time_end: time
    exit_time: time
    parameters: Dict[str, Any]
    description: str

print("✅ Core classes loaded successfully!")

## 📊 Data Engine

In [ ]:
class DataEngine:
    """Handles data loading, cleaning, and preparation."""
    
    def __init__(self, base_path: str):
        self.base_path = base_path
        
    def load_parquet_file(self, symbol: str, date: str = None) -> MarketData:
        """Load parquet file for a symbol."""
        try:
            file_path = f'{self.base_path}/parquet_files/{symbol}.parquet'
            if not os.path.exists(file_path):
                return self._create_sample_data(symbol)
                
            df = pd.read_parquet(file_path)
            df.index = pd.to_datetime(df.index)
            
            # Standardize column names
            df.columns = df.columns.str.lower()
            
            market_data = MarketData(symbol=symbol, data=df)
            if not market_data.validate():
                print(f"⚠️ Data validation failed for {symbol}, using sample data")
                return self._create_sample_data(symbol)
                
            return market_data
            
        except Exception as e:
            print(f"⚠️ Error loading {symbol}: {e}, using sample data")
            return self._create_sample_data(symbol)
    
    def _create_sample_data(self, symbol: str) -> MarketData:
        """Create sample intraday data for testing."""
        dates = pd.date_range('2024-01-01 09:15:00', '2024-01-01 15:30:00', freq='1min')
        
        # Generate realistic price movements
        np.random.seed(42)
        base_price = 1000.0
        returns = np.random.normal(0, 0.002, len(dates))
        prices = base_price * np.cumprod(1 + returns)
        
        # Create OHLCV
        data = pd.DataFrame({
            'open': prices,
            'high': prices * (1 + np.abs(np.random.normal(0, 0.005, len(dates)))),
            'low': prices * (1 - np.abs(np.random.normal(0, 0.005, len(dates)))),
            'close': prices,
            'volume': np.random.randint(1000, 10000, len(dates))
        }, index=dates)
        
        # Ensure high >= max(open, close) and low <= min(open, close)
        data['high'] = np.maximum(data['high'], np.maximum(data['open'], data['close']))
        data['low'] = np.minimum(data['low'], np.minimum(data['open'], data['close']))
        
        return MarketData(symbol=symbol, data=data)
    
    def get_available_symbols(self) -> List[str]:
        """Get list of available symbols from parquet files."""
        parquet_dir = f'{self.base_path}/parquet_files'
        if not os.path.exists(parquet_dir):
            return ['RELIANCE', 'TCS', 'HDFCBANK']  # Default sample symbols
        
        files = [f.replace('.parquet', '') for f in os.listdir(parquet_dir) 
                if f.endswith('.parquet')]
        return files if files else ['RELIANCE', 'TCS', 'HDFCBANK']

print("✅ Data Engine initialized!")

## 🎯 Strategy Engine

In [ ]:
class StrategyEngine:
    """Executes trading strategies on market data."""
    
    def __init__(self, data_engine: DataEngine):
        self.data_engine = data_engine
        self.strategies = {}
        
    def register_strategy(self, config: StrategyConfig, strategy_func):
        """Register a trading strategy."""
        self.strategies[config.name] = {
            'config': config,
            'function': strategy_func
        }
        
    def run_strategy(self, strategy_name: str, symbols: List[str]) -> StrategyResult:
        """Run a strategy on given symbols."""
        if strategy_name not in self.strategies:
            raise ValueError(f"Strategy {strategy_name} not found")
            
        strategy_info = self.strategies[strategy_name]
        config = strategy_info['config']
        strategy_func = strategy_info['function']
        
        all_trades = []
        
        for symbol in symbols:
            market_data = self.data_engine.load_parquet_file(symbol)
            trades = self._execute_strategy_on_symbol(
                market_data, config, strategy_func
            )
            all_trades.extend(trades)
        
        return self._calculate_strategy_result(strategy_name, all_trades)
    
    def _execute_strategy_on_symbol(self, market_data: MarketData, 
                                    config: StrategyConfig, strategy_func) -> List[Trade]:
        """Execute strategy on a single symbol."""
        df = market_data.data.copy()
        signals = strategy_func(df, config.parameters)
        
        trades = []
        position = None
        
        for timestamp, signal in signals.iterrows():
            current_time = timestamp.time()
            
            # Check exit time
            if position and current_time >= config.exit_time:
                exit_price = df.loc[timestamp, 'close']
                position.exit_time = timestamp
                position.exit_price = exit_price
                position.exit_reason = 'TIME_EXIT'
                trades.append(position)
                position = None
                
            # Check entry signal
            if signal['signal'] != 'HOLD' and not position:
                if config.entry_time_start <= current_time <= config.entry_time_end:
                    if config.side in ['BOTH', signal['signal']]:
                        entry_price = df.loc[timestamp, 'close']
                        position = Trade(
                            symbol=market_data.symbol,
                            strategy=config.name,
                            entry_time=timestamp,
                            exit_time=None,
                            entry_price=entry_price,
                            exit_price=None,
                            quantity=100,
                            side=signal['signal']
                        )
        
        return trades
    
    def _calculate_strategy_result(self, strategy_name: str, 
                                 trades: List[Trade]) -> StrategyResult:
        """Calculate strategy performance metrics."""
        if not trades:
            return StrategyResult(
                strategy_name=strategy_name,
                total_trades=0,
                winning_trades=0,
                losing_trades=0,
                total_pnl=0.0,
                max_drawdown=0.0,
                profit_factor=0.0,
                sharpe_ratio=0.0,
                avg_trade_duration=0.0,
                metadata={'message': 'No trades generated'}
            )
        
        pnls = [trade.pnl for trade in trades if trade.pnl is not None]
        winning_trades = sum(1 for pnl in pnls if pnl > 0)
        losing_trades = sum(1 for pnl in pnls if pnl < 0)
        
        total_pnl = sum(pnls)
        profit_factor = abs(sum(pnl for pnl in pnls if pnl > 0) / 
                          sum(pnl for pnl in pnls if pnl < 0)) if any(pnl < 0 for pnl in pnls) else float('inf')
        
        # Calculate durations
        durations = []
        for trade in trades:
            if trade.exit_time:
                duration = (trade.exit_time - trade.entry_time).total_seconds() / 60
                durations.append(duration)
        
        avg_duration = np.mean(durations) if durations else 0
        
        return StrategyResult(
            strategy_name=strategy_name,
            total_trades=len(trades),
            winning_trades=winning_trades,
            losing_trades=losing_trades,
            total_pnl=total_pnl,
            max_drawdown=min(pnls) if pnls else 0,
            profit_factor=profit_factor,
            sharpe_ratio=np.mean(pnls) / np.std(pnls) if len(pnls) > 1 and np.std(pnls) > 0 else 0,
            avg_trade_duration=avg_duration,
            metadata={'total_pnl_list': pnls}
        )

print("✅ Strategy Engine initialized!")

## 📈 Analytics Engine

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

class AnalyticsEngine:
    """Analyzes and visualizes strategy performance."""
    
    def __init__(self, base_path: str):
        self.base_path = base_path
        plt.style.use('default')
        sns.set_palette("husl")
        
    def analyze_strategy(self, result: StrategyResult) -> Dict[str, Any]:
        """Comprehensive strategy analysis."""
        analysis = {
            'strategy_name': result.strategy_name,
            'performance_metrics': {
                'total_trades': result.total_trades,
                'win_rate': result.winning_trades / result.total_trades if result.total_trades > 0 else 0,
                'total_pnl': result.total_pnl,
                'avg_trade_pnl': result.total_pnl / result.total_trades if result.total_trades > 0 else 0,
                'profit_factor': result.profit_factor,
                'sharpe_ratio': result.sharpe_ratio,
                'max_drawdown': result.max_drawdown,
                'avg_duration_min': result.avg_trade_duration
            },
            'quality_score': self._calculate_quality_score(result)
        }
        
        return analysis
    
    def _calculate_quality_score(self, result: StrategyResult) -> float:
        """Calculate overall strategy quality score (0-100)."""
        if result.total_trades == 0:
            return 0
            
        win_rate = result.winning_trades / result.total_trades
        profit_factor = min(result.profit_factor, 5)  # Cap at 5
        sharpe = max(min(result.sharpe_ratio, 3), -3)  # Cap between -3 and 3
        
        # Weighted score
        score = (
            win_rate * 30 +  # 30% weight
            min(profit_factor / 2, 1) * 40 +  # 40% weight (capped at 2.0)
            (sharpe + 3) / 6 * 30  # 30% weight (normalized -3 to 3 -> 0 to 1)
        )
        
        return round(score, 2)
    
    def plot_strategy_performance(self, result: StrategyResult):
        """Create performance visualization."""
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle(f'Strategy Performance: {result.strategy_name}', fontsize=16, fontweight='bold')
        
        # 1. PnL Distribution
        if 'total_pnl_list' in result.metadata and result.metadata['total_pnl_list']:
            pnls = result.metadata['total_pnl_list']
            axes[0, 0].hist(pnls, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
            axes[0, 0].axvline(0, color='red', linestyle='--', alpha=0.7)
            axes[0, 0].set_title('PnL Distribution')
            axes[0, 0].set_xlabel('PnL per Trade')
            axes[0, 0].set_ylabel('Frequency')
            axes[0, 0].grid(True, alpha=0.3)
        
        # 2. Performance Metrics
        metrics = ['Total Trades', 'Win Rate', 'Profit Factor', 'Sharpe Ratio']
        values = [
            result.total_trades,
            result.winning_trades / result.total_trades if result.total_trades > 0 else 0,
            min(result.profit_factor, 5),
            max(min(result.sharpe_ratio, 3), -3) + 3  # Normalize to 0-6
        ]
        
        bars = axes[0, 1].bar(metrics, values, color=['lightcoral', 'lightgreen', 'gold', 'plum'])
        axes[0, 1].set_title('Performance Metrics')
        axes[0, 1].tick_params(axis='x', rotation=45)
        
        # Add value labels on bars
        for bar, value in zip(bars, values):
            height = bar.get_height()
            axes[0, 1].text(bar.get_x() + bar.get_width()/2., height,
                          f'{value:.2f}', ha='center', va='bottom')
        
        # 3. Win/Loss Breakdown
        if result.total_trades > 0:
            win_pct = result.winning_trades / result.total_trades * 100
            loss_pct = result.losing_trades / result.total_trades * 100
            
            axes[1, 0].pie([win_pct, loss_pct], 
                         labels=['Winning', 'Losing'],
                         colors=['lightgreen', 'lightcoral'],
                         autopct='%1.1f%%',
                         startangle=90)
            axes[1, 0].set_title('Win/Loss Ratio')
        
        # 4. Summary Stats
        summary_text = f"""
        Strategy: {result.strategy_name}
        ━━━━━━━━━━━━━━━━━━━━━━━━━━━
        Total PnL: ₹{result.total_pnl:,.2f}
        Total Trades: {result.total_trades}
        Win Rate: {result.winning_trades / result.total_trades * 100:.1f}%
        Profit Factor: {result.profit_factor:.2f}
        Sharpe Ratio: {result.sharpe_ratio:.2f}
        Max Drawdown: ₹{result.max_drawdown:,.2f}
        Avg Duration: {result.avg_trade_duration:.1f} min
        ━━━━━━━━━━━━━━━━━━━━━━━━━━━
        Quality Score: {self._calculate_quality_score(result)}/100
        """
        
        axes[1, 1].text(0.1, 0.5, summary_text, fontsize=10, 
                       fontfamily='monospace', verticalalignment='center')
        axes[1, 1].set_xlim(0, 1)
        axes[1, 1].set_ylim(0, 1)
        axes[1, 1].axis('off')
        axes[1, 1].set_title('Summary Statistics')
        
        plt.tight_layout()
        plt.show()
    
    def compare_strategies(self, results: List[StrategyResult]):
        """Compare multiple strategies."""
        if not results:
            return
            
        fig, axes = plt.subplots(2, 3, figsize=(18, 10))
        fig.suptitle('Strategy Comparison', fontsize=16, fontweight='bold')
        
        names = [r.strategy_name for r in results]
        
        # 1. Total PnL
        axes[0, 0].bar(names, [r.total_pnl for r in results], color='skyblue')
        axes[0, 0].set_title('Total PnL')
        axes[0, 0].tick_params(axis='x', rotation=45)
        
        # 2. Win Rate
        win_rates = [r.winning_trades / r.total_trades if r.total_trades > 0 else 0 
                    for r in results]
        axes[0, 1].bar(names, win_rates, color='lightgreen')
        axes[0, 1].set_title('Win Rate')
        axes[0, 1].tick_params(axis='x', rotation=45)
        
        # 3. Profit Factor
        axes[0, 2].bar(names, [min(r.profit_factor, 5) for r in results], color='gold')
        axes[0, 2].set_title('Profit Factor (capped at 5)')
        axes[0, 2].tick_params(axis='x', rotation=45)
        
        # 4. Sharpe Ratio
        axes[1, 0].bar(names, [r.sharpe_ratio for r in results], color='plum')
        axes[1, 0].set_title('Sharpe Ratio')
        axes[1, 0].tick_params(axis='x', rotation=45)
        
        # 5. Total Trades
        axes[1, 1].bar(names, [r.total_trades for r in results], color='lightcoral')
        axes[1, 1].set_title('Total Trades')
        axes[1, 1].tick_params(axis='x', rotation=45)
        
        # 6. Quality Scores
        quality_scores = [self._calculate_quality_score(r) for r in results]
        axes[1, 2].bar(names, quality_scores, color='orange')
        axes[1, 2].set_title('Quality Score (/100)')
        axes[1, 2].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()

print("✅ Analytics Engine initialized!")

## 🎯 Example Trading Strategies

In [ ]:
def orb_strategy(df: pd.DataFrame, params: Dict[str, Any]) -> pd.DataFrame:
    """Opening Range Breakout Strategy."""
    
    # Extract parameters with defaults
    range_start = params.get('range_start', time(9, 15))
    range_end = params.get('range_end', time(9, 30))
    breakout_threshold = params.get('breakout_threshold', 0.002)
    
    signals = pd.DataFrame(index=df.index, columns=['signal'], data='HOLD')
    
    for date in df.index.date:
        daily_data = df[df.index.date == date]
        
        # Find opening range
        range_mask = (daily_data.index.time >= range_start) & (daily_data.index.time <= range_end)
        opening_range = daily_data[range_mask]
        
        if len(opening_range) < 2:
            continue
            
        range_high = opening_range['high'].max()
        range_low = opening_range['low'].min()
        
        # Check for breakouts after opening range
        post_range = daily_data[daily_data.index.time > range_end]
        
        for timestamp, row in post_range.iterrows():
            if row['high'] > range_high * (1 + breakout_threshold):
                signals.loc[timestamp] = 'LONG'
                break  # Only one trade per day
            elif row['low'] < range_low * (1 - breakout_threshold):
                signals.loc[timestamp] = 'SHORT'
                break
    
    return signals


def vwap_pullback_strategy(df: pd.DataFrame, params: Dict[str, Any]) -> pd.DataFrame:
    """VWAP Pullback Strategy."""
    
    lookback_period = params.get('lookback_period', 20)
    pullback_threshold = params.get('pullback_threshold', 0.001)
    
    signals = pd.DataFrame(index=df.index, columns=['signal'], data='HOLD')
    
    # Calculate VWAP
    df['vwap'] = (df['close'] * df['volume']).rolling(window=lookback_period).sum() / df['volume'].rolling(window=lookback_period).sum()
    
    for date in df.index.date:
        daily_data = df[df.index.date == date]
        
        for i in range(lookback_period, len(daily_data)):
            current_price = daily_data.iloc[i]['close']
            current_vwap = daily_data.iloc[i]['vwap']
            
            if pd.isna(current_vwap):
                continue
                
            # Long entry: price above VWAP and pulls back
            if current_price > current_vwap:
                recent_high = daily_data.iloc[i-5:i]['high'].max()
                if current_price < recent_high * (1 - pullback_threshold):
                    signals.loc[daily_data.index[i]] = 'LONG'
                    break
            
            # Short entry: price below VWAP and pulls back
            elif current_price < current_vwap:
                recent_low = daily_data.iloc[i-5:i]['low'].min()
                if current_price > recent_low * (1 + pullback_threshold):
                    signals.loc[daily_data.index[i]] = 'SHORT'
                    break
    
    return signals


def momentum_strategy(df: pd.DataFrame, params: Dict[str, Any]) -> pd.DataFrame:
    """Simple Momentum Strategy."""
    
    momentum_period = params.get('momentum_period', 10)
    momentum_threshold = params.get('momentum_threshold', 0.01)
    
    signals = pd.DataFrame(index=df.index, columns=['signal'], data='HOLD')
    
    # Calculate momentum
    df['momentum'] = df['close'].pct_change(momentum_period)
    
    for date in df.index.date:
        daily_data = df[df.index.date == date]
        
        for i in range(momentum_period, len(daily_data)):
            current_momentum = daily_data.iloc[i]['momentum']
            
            if pd.isna(current_momentum):
                continue
                
            if current_momentum > momentum_threshold:
                signals.loc[daily_data.index[i]] = 'LONG'
                break
            elif current_momentum < -momentum_threshold:
                signals.loc[daily_data.index[i]] = 'SHORT'
                break
    
    return signals

print("✅ Trading strategies loaded!")

## 🚀 Initialize ITOS System

In [ ]:
# Initialize all engines
data_engine = DataEngine(BASE_PATH)
strategy_engine = StrategyEngine(data_engine)
analytics_engine = AnalyticsEngine(BASE_PATH)

# Get available symbols
symbols = data_engine.get_available_symbols()
print(f"📊 Available symbols: {symbols}")

# Register strategies
strategies_to_register = [
    StrategyConfig(
        name='STR_001_ORB',
        side='BOTH',
        entry_time_start=time(9, 30),
        entry_time_end=time(15, 15),
        exit_time=time(15, 20),
        parameters={
            'range_start': time(9, 15),
            'range_end': time(9, 30),
            'breakout_threshold': 0.002
        },
        description='Opening Range Breakout Strategy'
    ),
    StrategyConfig(
        name='STR_002_VWAP_PULLBACK',
        side='BOTH',
        entry_time_start=time(9, 30),
        entry_time_end=time(15, 15),
        exit_time=time(15, 20),
        parameters={
            'lookback_period': 20,
            'pullback_threshold': 0.001
        },
        description='VWAP Pullback Strategy'
    ),
    StrategyConfig(
        name='STR_003_MOMENTUM',
        side='BOTH',
        entry_time_start=time(9, 30),
        entry_time_end=time(15, 15),
        exit_time=time(15, 20),
        parameters={
            'momentum_period': 10,
            'momentum_threshold': 0.01
        },
        description='Momentum Strategy'
    )
]

# Register strategies with their functions
strategy_engine.register_strategy(strategies_to_register[0], orb_strategy)
strategy_engine.register_strategy(strategies_to_register[1], vwap_pullback_strategy)
strategy_engine.register_strategy(strategies_to_register[2], momentum_strategy)

print("✅ ITOS System initialized successfully!")
print(f"📈 Registered strategies: {[s.name for s in strategies_to_register]}")

## 🧪 Run Strategy Tests

In [ ]:
# Test all strategies
print("🧪 Testing ITOS Strategies...\n")
print("="*50)

results = []

for strategy_config in strategies_to_register:
    print(f"\n🎯 Running {strategy_config.name}...")
    
    try:
        result = strategy_engine.run_strategy(strategy_config.name, symbols)
        results.append(result)
        
        # Print basic results
        print(f"   ✅ Total Trades: {result.total_trades}")
        print(f"   💰 Total PnL: ₹{result.total_pnl:,.2f}")
        print(f"   📊 Win Rate: {result.winning_trades / result.total_trades * 100:.1f}%" if result.total_trades > 0 else "   📊 Win Rate: 0%")
        print(f"   🎯 Quality Score: {analytics_engine._calculate_quality_score(result)}/100")
        
    except Exception as e:
        print(f"   ❌ Error: {str(e)}")

print("\n" + "="*50)
print("🎉 Strategy Testing Complete!")

## 📊 Performance Analysis

In [ ]:
# Compare all strategies
if results:
    print("📈 Strategy Comparison Analysis\n")
    
    # Create comparison table
    comparison_data = []
    for result in results:
        analysis = analytics_engine.analyze_strategy(result)
        metrics = analysis['performance_metrics']
        comparison_data.append({
            'Strategy': result.strategy_name,
            'Total Trades': metrics['total_trades'],
            'Win Rate %': f"{metrics['win_rate']*100:.1f}%",
            'Total PnL': f"₹{metrics['total_pnl']:,.0f}",
            'Profit Factor': f"{metrics['profit_factor']:.2f}",
            'Sharpe Ratio': f"{metrics['sharpe_ratio']:.2f}",
            'Quality Score': f"{analysis['quality_score']}/100"
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    display(comparison_df)
    
    # Plot comparison charts
    analytics_engine.compare_strategies(results)
    
    # Detailed analysis for each strategy
    for result in results:
        print(f"\n" + "="*60)
        print(f"📊 Detailed Analysis: {result.strategy_name}")
        print("="*60)
        
        analytics_engine.plot_strategy_performance(result)
else:
    print("❌ No strategy results to analyze")

## 💾 Save Results

In [ ]:
# Save results to Google Drive
if results:
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Save detailed results
    results_data = []
    for result in results:
        analysis = analytics_engine.analyze_strategy(result)
        results_data.append({
            'timestamp': timestamp,
            'strategy_name': result.strategy_name,
            'total_trades': result.total_trades,
            'winning_trades': result.winning_trades,
            'losing_trades': result.losing_trades,
            'total_pnl': result.total_pnl,
            'max_drawdown': result.max_drawdown,
            'profit_factor': result.profit_factor,
            'sharpe_ratio': result.sharpe_ratio,
            'avg_trade_duration': result.avg_trade_duration,
            'quality_score': analysis['quality_score']
        })
    
    results_df = pd.DataFrame(results_data)
    results_file = f'{BASE_PATH}/results/strategy_results_{timestamp}.csv'
    results_df.to_csv(results_file, index=False)
    
    # Save comparison table
    comparison_df.to_csv(f'{BASE_PATH}/results/strategy_comparison_{timestamp}.csv', index=False)
    
    print(f"💾 Results saved to Google Drive:")
    print(f"   📄 Detailed results: {results_file}")
    print(f"   📊 Comparison table: {BASE_PATH}/results/strategy_comparison_{timestamp}.csv")
    
    # Save to current session for download
    results_df.to_csv('/content/strategy_results.csv', index=False)
    comparison_df.to_csv('/content/strategy_comparison.csv', index=False)
    
    print(f"\n📥 Files ready for download in Colab session:")
    print(f"   📄 /content/strategy_results.csv")
    print(f"   📊 /content/strategy_comparison.csv")
else:
    print("❌ No results to save")

## 🎯 Next Steps

### **What you can do now:**

1. **📁 Upload Real Data**:
   - Upload your Parquet files to `/content/drive/MyDrive/itos_data/parquet_files/`
   - Make sure files are named `{SYMBOL}.parquet` (e.g., `RELIANCE.parquet`)

2. **🎯 Add New Strategies**:
   ```python
   # Example: Add your custom strategy
   def my_custom_strategy(df, params):
       # Your logic here
       return signals
   
   my_config = StrategyConfig(
       name='MY_CUSTOM_STRATEGY',
       side='BOTH',
       entry_time_start=time(9, 30),
       entry_time_end=time(15, 15),
       exit_time=time(15, 20),
       parameters={'param1': value1},
       description='My custom strategy'
   )
   strategy_engine.register_strategy(my_config, my_custom_strategy)
   ```

3. **🔧 Optimize Parameters**:
   - Test different parameter values for each strategy
   - Use the quality score to find optimal settings

4. **📈 Scale to More Symbols**:
   - Add more symbols to your parquet files
   - Test strategies across different market conditions

### **System Features:**

- ✅ **Complete ITOS framework** in one notebook
- ✅ **3 working strategies** with customizable parameters
- ✅ **Comprehensive analytics** with visualizations
- ✅ **Real data support** (Parquet files)
- ✅ **Performance tracking** and quality scoring
- ✅ **Google Drive integration** for saving results

### **🚀 Ready to scale to 500+ strategies!**

This single notebook contains everything you need to build, test, and deploy your intraday trading strategies. Just add your data and start experimenting!